# Imports

In [146]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from collections import defaultdict
import os

# Années précédentes

In [73]:
years=[i for i in range(2011,2025)]
years

[2011,
 2012,
 2013,
 2014,
 2015,
 2016,
 2017,
 2018,
 2019,
 2020,
 2021,
 2022,
 2023,
 2024]

In [74]:
url_list=[f'https://fr.wikipedia.org/wiki/Miss_France_{year}' for year in years]
url_list

['https://fr.wikipedia.org/wiki/Miss_France_2011',
 'https://fr.wikipedia.org/wiki/Miss_France_2012',
 'https://fr.wikipedia.org/wiki/Miss_France_2013',
 'https://fr.wikipedia.org/wiki/Miss_France_2014',
 'https://fr.wikipedia.org/wiki/Miss_France_2015',
 'https://fr.wikipedia.org/wiki/Miss_France_2016',
 'https://fr.wikipedia.org/wiki/Miss_France_2017',
 'https://fr.wikipedia.org/wiki/Miss_France_2018',
 'https://fr.wikipedia.org/wiki/Miss_France_2019',
 'https://fr.wikipedia.org/wiki/Miss_France_2020',
 'https://fr.wikipedia.org/wiki/Miss_France_2021',
 'https://fr.wikipedia.org/wiki/Miss_France_2022',
 'https://fr.wikipedia.org/wiki/Miss_France_2023',
 'https://fr.wikipedia.org/wiki/Miss_France_2024']

In [75]:
# Dictionnaire pour stocker les DataFrames
dataframes = {}

In [76]:
def extract_candidates_table(url, year):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Localiser la section "Candidates"
    section_candidates = None
    for header in soup.find_all('h2'):
        if "Candidates" in header.text:
            section_candidates = header
            break
    
    if section_candidates:
        # Trouver la table qui suit immédiatement cette section
        candidates_table = section_candidates.find_next('table', {'class': 'wikitable'})
        if candidates_table:
            # Extraire les en-têtes de la table
            headers = [header.text.strip() for header in candidates_table.find_all('th')]
            # Extraire les lignes de la table
            rows = []
            for row in candidates_table.find_all('tr')[1:]:  # Ignorer les en-têtes
                cols = [col.text.strip() for col in row.find_all('td')]
                rows.append(cols)
            # Créer un DataFrame pandas
            df = pd.DataFrame(rows, columns=headers)
            df['année'] = year
            return df
    return None

In [77]:
for url in url_list:
    year = url.split('_')[-1]  # Extraire l'année de l'URL
    df = extract_candidates_table(url, year)
    if df is not None:
        dataframes[f"année {year}"] = df
        print(f"Données extraites pour l'année {year}.")
    else:
        print(f"Pas de table 'Candidates' trouvée pour l'année {year}.")

Données extraites pour l'année 2011.
Données extraites pour l'année 2012.
Données extraites pour l'année 2013.
Données extraites pour l'année 2014.
Données extraites pour l'année 2015.
Données extraites pour l'année 2016.
Données extraites pour l'année 2017.
Données extraites pour l'année 2018.
Données extraites pour l'année 2019.
Données extraites pour l'année 2020.
Données extraites pour l'année 2021.
Données extraites pour l'année 2022.
Données extraites pour l'année 2023.
Données extraites pour l'année 2024.


In [78]:
print(dataframes.keys())

dict_keys(['année 2011', 'année 2012', 'année 2013', 'année 2014', 'année 2015', 'année 2016', 'année 2017', 'année 2018', 'année 2019', 'année 2020', 'année 2021', 'année 2022', 'année 2023', 'année 2024'])


In [79]:
categories = defaultdict(list)

# Identifier les headers de chaque DataFrame et les regrouper
for year, df in dataframes.items():
    headers = tuple(df.columns)  # Les colonnes de chaque DataFrame sous forme de tuple
    categories[headers].append(year)

# Afficher les catégories et les années associées
for idx, (header_set, years) in enumerate(categories.items(), start=1):
    print(f"Catégorie {idx}:")
    print("Headers :", header_set)
    print("Années :", years)
    print("-" * 50)


Catégorie 1:
Headers : ('Région', 'Nom', 'Âge', 'Taille', 'Résidence', 'Élue le', 'Classement final', 'année')
Années : ['année 2011', 'année 2014', 'année 2015', 'année 2016', 'année 2021']
--------------------------------------------------
Catégorie 2:
Headers : ('Région', 'Nom[2]', 'Âge', 'Taille', 'Résidence', 'Élue le', 'Classement final', 'année')
Années : ['année 2012', 'année 2013']
--------------------------------------------------
Catégorie 3:
Headers : ('Région', 'Nom', 'Âge', 'Taille', 'Résidence', 'Élue le', 'Résultat', 'année')
Années : ['année 2017']
--------------------------------------------------
Catégorie 4:
Headers : ('région', 'Nom', 'Âge', 'Taille', 'Résidence', 'Élue le', 'Groupe', 'Classement final', 'année')
Années : ['année 2018']
--------------------------------------------------
Catégorie 5:
Headers : ('région', 'Nom', 'Âge', 'Taille', 'Résidence', 'Élue le', 'Classement final', 'année')
Années : ['année 2019']
----------------------------------------------

# Nettoyage des colonnes

In [80]:
def clean_columns(df):
    df.columns = df.columns.str.replace(r'\[.*?\]', '', regex=True).str.strip()
    df.columns = df.columns.str.replace(r'^région$', 'Région', regex=True)
    df.columns = df.columns.str.replace(r'Élue le','Date élection', regex=True)
    df.columns = df.columns.str.replace(r'Élection régionale','Date élection', regex=True)
    df.columns = df.columns.str.replace(r'final$','Final', regex=True)
    df.columns = df.columns.str.replace(r'Résultat','Classement Final', regex=True)
    if 'Réf.' in df.columns:
        df = df.drop(columns=['Réf.'])
    if 'Groupe' in df.columns:
        df = df.drop(columns=['Groupe'])
    return df

for year, df in dataframes.items():
    dataframes[year] = clean_columns(df)

for year, df in dataframes.items():
    print(f"Année : {year}")
    print(df.columns)
    print("-" * 50)

Année : année 2011
Index(['Région', 'Nom', 'Âge', 'Taille', 'Résidence', 'Date élection',
       'Classement Final', 'année'],
      dtype='object')
--------------------------------------------------
Année : année 2012
Index(['Région', 'Nom', 'Âge', 'Taille', 'Résidence', 'Date élection',
       'Classement Final', 'année'],
      dtype='object')
--------------------------------------------------
Année : année 2013
Index(['Région', 'Nom', 'Âge', 'Taille', 'Résidence', 'Date élection',
       'Classement Final', 'année'],
      dtype='object')
--------------------------------------------------
Année : année 2014
Index(['Région', 'Nom', 'Âge', 'Taille', 'Résidence', 'Date élection',
       'Classement Final', 'année'],
      dtype='object')
--------------------------------------------------
Année : année 2015
Index(['Région', 'Nom', 'Âge', 'Taille', 'Résidence', 'Date élection',
       'Classement Final', 'année'],
      dtype='object')
--------------------------------------------------


In [81]:
categories = defaultdict(list)

for year, df in dataframes.items():
    headers = tuple(df.columns)  
    categories[headers].append(year)

for idx, (header_set, years) in enumerate(categories.items(), start=1):
    print(f"Catégorie {idx}:")
    print("Headers :", header_set)
    print("Années :", years)
    print("-" * 50)

Catégorie 1:
Headers : ('Région', 'Nom', 'Âge', 'Taille', 'Résidence', 'Date élection', 'Classement Final', 'année')
Années : ['année 2011', 'année 2012', 'année 2013', 'année 2014', 'année 2015', 'année 2016', 'année 2017', 'année 2018', 'année 2019', 'année 2020', 'année 2021', 'année 2022', 'année 2023', 'année 2024']
--------------------------------------------------


In [82]:
df_miss = pd.concat(dataframes.values(), ignore_index=True)

print("DataFrame fusionnée :")
print(df_miss.head())
print(f"Taille de la DataFrame fusionnée : {df_miss.shape}")

DataFrame fusionnée :
           Région                 Nom     Âge  Taille         Résidence  \
0     Miss Alsace    Mathilde Buecher  19 ans  1,78 m       Heimersdorf   
1  Miss Aquitaine      Clémence Thill  20 ans  1,75 m              Agen   
2   Miss Auvergne     Clémence Oleksy  19 ans  1,76 m             Vichy   
3  Miss Bourgogne  Alice Detollenaere  23 ans  1,74 m  Chitry-les-Mines   
4   Miss Bretagne     Laury Thilleman  19 ans  1,78 m             Brest   

                         Date élection  \
0          10 octobre 2010 à Hirsingue   
1           16 octobre 2010 à Cadillac   
2            17 octobre 2010 à Cébazat   
3  1er octobre 2010 à Chalon-sur-Saône   
4            26 octobre 2010 à Pontivy   

                                Classement Final année  
0                                                 2011  
1                                                 2011  
2                                    2e dauphine  2011  
3                            Prix du Mannequin

# Transformation des données

## Données catégoriques

In [83]:
categorical_columns = ['Région']

for col in categorical_columns:
    unique_values = df_miss[col].unique()
    print(f"Valeurs uniques dans la colonne '{col}':")
    print(unique_values)
    print("-" * 50)

Valeurs uniques dans la colonne 'Région':
['Miss Alsace' 'Miss Aquitaine' 'Miss Auvergne' 'Miss Bourgogne'
 'Miss Bretagne' 'Miss Centre' 'Miss Champagne-Ardenne' 'Miss Corse'
 "Miss Côte d'Azur" 'Miss Franche-Comté' 'Miss Guadeloupe' 'Miss Guyane'
 'Miss Île-de-France' 'Miss Languedoc' 'Miss Limousin' 'Miss Lorraine'
 'Miss Martinique' 'Miss Mayotte' 'Miss Midi-Pyrénées'
 'Miss Nord-Pas-de-Calais' 'Miss Normandie' 'Miss Nouvelle-Calédonie'
 'Miss Orléanais' 'Miss Pays de Loire' 'Miss Pays de Savoie'
 'Miss Picardie' 'Miss Poitou-Charentes' 'Miss Provence' 'Miss Réunion'
 'Miss Rhône-Alpes' 'Miss Roussillon' 'Miss Saint-Pierre-et-Miquelon'
 'Miss Tahiti' 'Miss Saint-Martin' 'Miss Centre-Val-de-Loire'
 'Miss Guadeloupe et Îles du Nord' 'Miss Centre-Val de Loire'
 'Miss Languedoc-Roussillon' 'Miss Saint-Martin et Saint-Barthélemy'
 'Miss Pays de la Loire' 'Miss Wallis-et-Futuna'
 'Miss Saint-Martin et Saint-Barthélémy']
--------------------------------------------------


In [84]:
cleaning_dict = {
    "Miss Saint-Martin et Saint-Barthélémy": "Miss Saint-Martin et Saint-Barthélemy",
    "Miss Saint-Martin et Saint-Barthélemy": "Miss Saint-Martin et Saint-Barthélemy",
    "Miss Centre-Val de Loire": "Miss Centre-Val-de-Loire",
    "Miss Pays de Loire": "Miss Pays de la Loire",
    "Miss Guadeloupe et Îles du Nord": "Miss Guadeloupe",
    "Miss Wallis-et-Futuna": "Miss Wallis et Futuna",
    "Miss Saint-Martin": "Miss Saint-Martin et Saint-Barthélemy",
}

df_miss['Région'] = df_miss['Région'].replace(cleaning_dict)

unique_regions = df_miss['Région'].unique()
print("Valeurs uniques après nettoyage :")
print(unique_regions)

Valeurs uniques après nettoyage :
['Miss Alsace' 'Miss Aquitaine' 'Miss Auvergne' 'Miss Bourgogne'
 'Miss Bretagne' 'Miss Centre' 'Miss Champagne-Ardenne' 'Miss Corse'
 "Miss Côte d'Azur" 'Miss Franche-Comté' 'Miss Guadeloupe' 'Miss Guyane'
 'Miss Île-de-France' 'Miss Languedoc' 'Miss Limousin' 'Miss Lorraine'
 'Miss Martinique' 'Miss Mayotte' 'Miss Midi-Pyrénées'
 'Miss Nord-Pas-de-Calais' 'Miss Normandie' 'Miss Nouvelle-Calédonie'
 'Miss Orléanais' 'Miss Pays de la Loire' 'Miss Pays de Savoie'
 'Miss Picardie' 'Miss Poitou-Charentes' 'Miss Provence' 'Miss Réunion'
 'Miss Rhône-Alpes' 'Miss Roussillon' 'Miss Saint-Pierre-et-Miquelon'
 'Miss Tahiti' 'Miss Saint-Martin et Saint-Barthélemy'
 'Miss Centre-Val-de-Loire' 'Miss Languedoc-Roussillon'
 'Miss Wallis et Futuna']


In [85]:
df_miss

,Région,Nom,Âge,Taille,Résidence,Date élection,Classement Final,année
0,Miss Alsace,Mathilde Buecher,19 ans,"1,78 m",Heimersdorf,10 octobre 2010 à Hirsingue,,2011
1,Miss Aquitaine,Clémence Thill,20 ans,"1,75 m",Agen,16 octobre 2010 à Cadillac,,2011
2,Miss Auvergne,Clémence Oleksy,19 ans,"1,76 m",Vichy,17 octobre 2010 à Cébazat,2e dauphine,2011
3,Miss Bourgogne,Alice Detollenaere,23 ans,"1,74 m",Chitry-les-Mines,1er octobre 2010 à Chalon-sur-Saône,Prix du Mannequinat,2011
4,Miss Bretagne,Laury Thilleman,19 ans,"1,78 m",Brest,26 octobre 2010 à Pontivy,Miss France 2011\nPrix de la Culture générale,2011
...,...,...,...,...,...,...,...,...
429,Miss Provence,Adélina Blanc,25 ans,"1,73 m",Eyragues,27 juillet 2023 à Saint-Raphaël,2e dauphine,2024
430,Miss Réunion,Mélanie Odules,20 ans,"1,78 m",Saint-Paul,26 août 2023 à Saint-Denis,,2024
431,Miss Rhône-Alpes,Alizée Bidaut,22 ans,"1,74 m",Saint-Genis-sur-Menthon,23 septembre 2023 à La Roche-sur-Foron,,2024
432,Miss Roussillon,Élise Aquilina,21 ans,"1,71 m",Cabestany,5 août 2023 au Barcarès,Top 15 (11e)\nPrix de l’académie des Miss[25],2024


#### Nettoyer la colonne Âge

In [86]:
df_miss['Âge'] = df_miss['Âge'].str.extract(r'(\d+)').astype(int)

#### Nettoyer la colonne Taille

In [87]:
df_miss['Taille'] = df_miss['Taille'].str.replace(',', '.').str.extract(r'(\d+\.\d+)').astype(float)

In [88]:
import locale

# Gérer les mois en français
locale.setlocale(locale.LC_TIME, 'fr_FR.UTF-8')

'fr_FR.UTF-8'

#### Nettoyer la colonne Date élection

In [89]:
month_map = {
    "janvier": "01",
    "février": "02",
    "mars": "03",
    "avril": "04",
    "mai": "05",
    "juin": "06",
    "juillet": "07",
    "août": "08",
    "septembre": "09",
    "octobre": "10",
    "novembre": "11",
    "décembre": "12"
}

In [90]:
def replace_months(date_str):
    if pd.isna(date_str):  # Vérifier les valeurs manquantes
        return None
    for month, num in month_map.items():
        if month in date_str:
            return date_str.replace(month, num)
    return date_str

In [91]:
df_miss['Date élection'] = df_miss['Date élection'].str.extract(r'(\d{1,2} \w+ \d{4})')
df_miss['Date élection'] = df_miss['Date élection'].apply(replace_months)
print(df_miss)

df_miss['Date élection'] = pd.to_datetime(df_miss['Date élection'], format='%d %m %Y')

print(df_miss)

               Région                 Nom  Âge  Taille  \
0         Miss Alsace    Mathilde Buecher   19    1.78   
1      Miss Aquitaine      Clémence Thill   20    1.75   
2       Miss Auvergne     Clémence Oleksy   19    1.76   
3      Miss Bourgogne  Alice Detollenaere   23    1.74   
4       Miss Bretagne     Laury Thilleman   19    1.78   
..                ...                 ...  ...     ...   
429     Miss Provence       Adélina Blanc   25    1.73   
430      Miss Réunion      Mélanie Odules   20    1.78   
431  Miss Rhône-Alpes       Alizée Bidaut   22    1.74   
432   Miss Roussillon      Élise Aquilina   21    1.71   
433       Miss Tahiti    Ravahere Silloux   25    1.72   

                   Résidence Date élection  \
0                Heimersdorf    10 10 2010   
1                       Agen    16 10 2010   
2                      Vichy    17 10 2010   
3           Chitry-les-Mines          None   
4                      Brest    26 10 2010   
..                       ..

In [92]:
missing_dates = df_miss[df_miss['Date élection'].isna()]

print("Miss sans date d'élection :")
print(missing_dates)

Miss sans date d'élection :
                        Région                  Nom  Âge  Taille  \
3               Miss Bourgogne   Alice Detollenaere   23    1.74   
44                 Miss Guyane      Anaële Veilleur   19    1.73   
48               Miss Lorraine            Maud Pisa   21    1.76   
73                  Miss Corse    Louise Robert [5]   22    1.80   
83                Miss Mayotte        Stanisla Saïd   22    1.76   
87     Miss Nouvelle-Calédonie        Sandra Bergès   18    1.73   
178             Miss Languedoc  Lena Stachurski[24]   20    1.74   
218      Miss Pays de la Loire         Carla Loones   21    1.77   
329  Miss Languedoc-Roussillon     Illana Barry[46]   19    1.76   
358  Miss Languedoc-Roussillon     Marion Ratié[41]   20    1.72   
359              Miss Limousin       Julie Beve[42]   23    1.71   
388              Miss Limousin          Salomé Maud   23    1.74   
418              Miss Limousin      Agathe Toullieu   22    1.77   
428      Miss Poitou

In [93]:
region_mean_dates = df_miss.groupby('Région')['Date élection'].transform('mean')

df_miss['Date élection'] = df_miss['Date élection'].fillna(region_mean_dates)

# Afficher le DataFrame mis à jour
print("DataFrame avec valeurs manquantes remplacées :")
df_miss

DataFrame avec valeurs manquantes remplacées :


,Région,Nom,Âge,Taille,Résidence,Date élection,Classement Final,année
0,Miss Alsace,Mathilde Buecher,19,1.78,Heimersdorf,2010-10-10,,2011
1,Miss Aquitaine,Clémence Thill,20,1.75,Agen,2010-10-16,,2011
2,Miss Auvergne,Clémence Oleksy,19,1.76,Vichy,2010-10-17,2e dauphine,2011
3,Miss Bourgogne,Alice Detollenaere,23,1.74,Chitry-les-Mines,2017-09-29,Prix du Mannequinat,2011
4,Miss Bretagne,Laury Thilleman,19,1.78,Brest,2010-10-26,Miss France 2011\nPrix de la Culture générale,2011
...,...,...,...,...,...,...,...,...
429,Miss Provence,Adélina Blanc,25,1.73,Eyragues,2023-07-27,2e dauphine,2024
430,Miss Réunion,Mélanie Odules,20,1.78,Saint-Paul,2023-08-26,,2024
431,Miss Rhône-Alpes,Alizée Bidaut,22,1.74,Saint-Genis-sur-Menthon,2023-09-23,,2024
432,Miss Roussillon,Élise Aquilina,21,1.71,Cabestany,2023-08-05,Top 15 (11e)\nPrix de l’académie des Miss[25],2024


# Résultats top 15

## Résultats 2024

## Résultats 2011 à 2021

In [94]:
results = {}
urls=url_list[0:11]
# Fonction pour scrapper les tableaux de la catégorie "Deuxième tour"
def scrape_second_round_table(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    # Identifier la section "Deuxième tour"
    section_second_round = None
    for header in soup.find_all(['h2', 'h3','h4','h5','h6']):
        if "Deuxième tour" in header.text:
            section_second_round = header
            break

    # Vérifier si la section a été trouvée
    if section_second_round:
        # Trouver le tableau dans cette section
        table = section_second_round.find_next('table', {'class': 'wikitable'})
        
        # Vérifier si un tableau a été trouvé
        if table:
            # Extraire les en-têtes et les lignes
            headers = [header.text.strip() for header in table.find_all('th')]
            rows = []
            for row in table.find_all('tr')[1:]:  # Ignorer les en-têtes
                cols = [col.text.strip() for col in row.find_all('td')]
                rows.append(cols)

            # Créer un DataFrame pandas
            if rows:
                df = pd.DataFrame(rows, columns=headers)
                return df
    else:
        print(f"Aucune section 'Deuxième tour' trouvée sur {url}")
        return None

for url in urls:
    year = url.split('_')[-1]
    print(f"Scraping {year}...")
    df = scrape_second_round_table(url)
    if df is not None:
        results[year] = df

# Résumé des résultats trouvés
for year, df in results.items():
    print(f"Année {year} : Tableau du deuxième tour extrait.")

Scraping 2011...
Scraping 2012...
Scraping 2013...
Scraping 2014...
Scraping 2015...
Scraping 2016...
Scraping 2017...
Scraping 2018...
Scraping 2019...
Scraping 2020...
Scraping 2021...
Année 2011 : Tableau du deuxième tour extrait.
Année 2012 : Tableau du deuxième tour extrait.
Année 2013 : Tableau du deuxième tour extrait.
Année 2014 : Tableau du deuxième tour extrait.
Année 2015 : Tableau du deuxième tour extrait.
Année 2016 : Tableau du deuxième tour extrait.
Année 2017 : Tableau du deuxième tour extrait.
Année 2018 : Tableau du deuxième tour extrait.
Année 2019 : Tableau du deuxième tour extrait.
Année 2020 : Tableau du deuxième tour extrait.
Année 2021 : Tableau du deuxième tour extrait.


## Résultats en 2022

In [95]:
url=url_list[11]
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

first_second_round = None
for header in soup.find_all(['h2', 'h3','h4','h5','h6']):
    if "Premier tour" in header.text:
        first_second_round = header
        break

if first_second_round:
    table = first_second_round.find_next('table', {'class': 'wikitable'})
    if table:
        headers = [header.text.strip() for header in table.find_all('th')]
        rows = []
        for row in table.find_all('tr')[1:]:  # Ignorer les en-têtes
            cols = [col.text.strip() for col in row.find_all('td')]
            rows.append(cols)
        if rows:
            df_results_2022 = pd.DataFrame(rows, columns=headers)

else:
    print(f"Aucune section 'Premier tour' trouvée sur {url}")

year = url.split('_')[-1]
print(f"Scraping {year}...")

Scraping 2022...


In [96]:
df_results_2022['Miss'] = df_results_2022['Miss'].str.replace(r'\[.*?\]', '', regex=True).str.strip()

In [97]:
df_results_2022

,Miss,Public,Jury,Total
0,Miss Île-de-France,14,15,29
1,Miss Tahiti,11,15,26
2,Miss Martinique,15,9,24
3,Miss Normandie,10,13,23
4,Miss Alsace,12,9,21
5,Miss Aquitaine,8,12,20
6,Miss Réunion,7,12,19
7,Miss Nord-Pas-de-Calais,6,12,18
8,Miss Rhône-Alpes,13,4,17
9,Miss Côte d'Azur,9,7,16


## Résultats en 2023

In [98]:
url=url_list[12]
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

classement = None
classement_sections = []
for header in soup.find_all(['h1', 'h2', 'h3', 'h4', 'h5', 'h6']):
    if "Classement" in header.text:
        classement_sections.append(header)

# Parcourir les sections pour identifier celle avec un tableau valide
df_results_2023 = None
for section in classement_sections:
    table = section.find_next('table', {'class': 'wikitable'})

    if table:
        # Extraire les en-têtes
        headers = [header.text.strip() for header in table.find_all('th')]

        # Extraire les lignes du tableau
        rows = []
        for row in table.find_all('tr')[1:]:  # Ignorer la ligne d'en-tête
            cols = [col.text.strip() for col in row.find_all('td')]
            rows.append(cols)

        # Vérifier les dimensions et ajuster si nécessaire
        adjusted_rows = [cols for cols in rows if len(cols) == len(headers)]

        if adjusted_rows:
            # Créer un DataFrame
            df_results_2023 = pd.DataFrame(adjusted_rows, columns=headers)
            print(f"Tableau extrait de la section : '{section.text.strip()}'")
            break  # Arrêter dès qu'un tableau valide est trouvé

if df_results_2023 is not None:
    print(f"Tableau extrait pour 2023 : {df_results_2023.shape[0]} lignes, {df_results_2023.shape[1]} colonnes")
    print(df_results_2023)
else:
    print("Aucun tableau valide trouvé pour 2023.")

Tableau extrait de la section : 'Classement'
Tableau extrait pour 2023 : 15 lignes, 4 colonnes
                       Miss Public Jury Total
0           Miss Guadeloupe     15   15    30
1   Miss Nord-Pas-de-Calais     12   15    27
2             Miss Auvergne     11   12    23
3           Miss Martinique     14    9    23
4        Miss Franche-Comté     13    9    22
5          Miss Côte d'Azur      7   13    20
6             Miss Lorraine     10    9    19
7        Miss Midi-Pyrénées      6   12    18
8             Miss Picardie      8    6    14
9            Miss Languedoc      1   12    13
10         Miss Rhône-Alpes      5    6    11
11    Miss Pays de la Loire      9    2    11
12          Miss Roussillon      4    6    10
13    Miss Poitou-Charentes      3    6     9
14           Miss Aquitaine      2    2     4


## Résultat en 2024

In [99]:
# URL de la page Wikipédia
url = "https://fr.wikipedia.org/wiki/Miss_France_2024"

# Charger la page
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

# Identifier la section "Résultats du Top 15"
section_results = None
for header in soup.find_all(['h2', 'h3']):
    if "Résultats du Top 15" in header.text:
        section_results = header
        break

# Vérifier si la section a été trouvée
if section_results:
    # Trouver le tableau qui suit cette section
    table = section_results.find_next('table', {'class': 'wikitable'})

    # Vérifier si un tableau a été trouvé
    if table:
        # Extraire les en-têtes de la table
        headers = [header.text.strip() for header in table.find_all('th')]

        # Extraire les lignes de données
        rows = []
        for row in table.find_all('tr')[1:]:  # Ignorer les en-têtes
            cols = [col.text.strip() for col in row.find_all('td')]
            rows.append(cols)

        # Créer un DataFrame pandas
        df_results_2024 = pd.DataFrame(rows, columns=headers)

        # Afficher un aperçu du DataFrame
        print(df_results_2024)

    else:
        print("Aucun tableau trouvé dans la section 'Résultats du Top 15'.")
else:
    print("Section 'Résultats du Top 15' non trouvée.")

                        Miss Public Jury Total
0    Miss Nord-Pas-de-Calais     13   15    28
1                Miss Guyane     15   11    26
2             Miss Languedoc     11   14    25
3            Miss Guadeloupe     14   11    25
4              Miss Provence     12   11    23
5             Miss Normandie      9   13    22
6                Miss Tahiti      8    8    16
7         Miss Île-de-France      2   13    15
8      Miss Pays de la Loire     10    4    14
9             Miss Bourgogne      5    8    13
10           Miss Roussillon      6    6    12
11          Miss Côte d'Azur      7    4    11
12  Miss Centre-Val de Loire      3    6     9
13                Miss Corse      4    4     8
14        Miss Midi-Pyrénées      1    4     5


In [100]:
# Ajouter les DataFrames des années 2022, 2023, et 2024 au dictionnaire
results["2022"] = df_results_2022
results["2023"] = df_results_2023
results["2024"] = df_results_2024

# Afficher un résumé du dictionnaire mis à jour
for year, df in results.items():
    print(f"Année {year} : {df.shape[0]} lignes, {df.shape[1]} colonnes")

Année 2011 : 12 lignes, 4 colonnes
Année 2012 : 12 lignes, 4 colonnes
Année 2013 : 12 lignes, 4 colonnes
Année 2014 : 12 lignes, 4 colonnes
Année 2015 : 12 lignes, 4 colonnes
Année 2016 : 12 lignes, 4 colonnes
Année 2017 : 12 lignes, 4 colonnes
Année 2018 : 12 lignes, 4 colonnes
Année 2019 : 12 lignes, 4 colonnes
Année 2020 : 15 lignes, 4 colonnes
Année 2021 : 15 lignes, 4 colonnes
Année 2022 : 15 lignes, 4 colonnes
Année 2023 : 15 lignes, 4 colonnes
Année 2024 : 15 lignes, 4 colonnes


# Tagging top 12/15 and jury points and public points

In [101]:
results

{'2011':                Miss[2] Public Jury Total
 0        Miss Bretagne     12   11    23
 1        Miss Auvergne     10   12    22
 2        Miss Picardie      9   10    19
 3       Miss Languedoc     11    6    17
 4   Miss Île-de-France      7    9    16
 5        Miss Provence      6    9    15
 6      Miss Martinique      8    5    13
 7       Miss Orléanais      1    9    10
 8      Miss Roussillon      5    5    10
 9           Miss Corse      3    5     8
 10      Miss Normandie      4    2     6
 11    Miss Côte d'Azur      2    2     4,
 '2012':                   Miss Public Jury Total
 0         Miss Réunion     10   10    20
 1          Miss Alsace     11    8    19
 2     Miss Côte d'Azur      6   12    18
 3   Miss Pays de Loire     12    6    18
 4        Miss Provence      2   12    14
 5      Miss Martinique      5    9    14
 6       Miss Languedoc      9    4    13
 7          Miss Guyane      7    5    12
 8        Miss Bretagne      8    4    12
 9      Miss Rous

In [102]:
for year, df_results in results.items():
    if 'Miss[2]' in df_results.columns:
        df_results.rename(columns={'Miss[2]': 'Miss'}, inplace=True)

results

{'2011':                   Miss Public Jury Total
 0        Miss Bretagne     12   11    23
 1        Miss Auvergne     10   12    22
 2        Miss Picardie      9   10    19
 3       Miss Languedoc     11    6    17
 4   Miss Île-de-France      7    9    16
 5        Miss Provence      6    9    15
 6      Miss Martinique      8    5    13
 7       Miss Orléanais      1    9    10
 8      Miss Roussillon      5    5    10
 9           Miss Corse      3    5     8
 10      Miss Normandie      4    2     6
 11    Miss Côte d'Azur      2    2     4,
 '2012':                   Miss Public Jury Total
 0         Miss Réunion     10   10    20
 1          Miss Alsace     11    8    19
 2     Miss Côte d'Azur      6   12    18
 3   Miss Pays de Loire     12    6    18
 4        Miss Provence      2   12    14
 5      Miss Martinique      5    9    14
 6       Miss Languedoc      9    4    13
 7          Miss Guyane      7    5    12
 8        Miss Bretagne      8    4    12
 9      Miss Rous

In [103]:
df_miss['Top 15/12'] = False
df_miss['Points_du_public'] = 0
df_miss['Points_du_jury'] = 0
df_miss['Points_totaux'] = 0

In [104]:
# Dictionnaire de mapping
mapping = {
    "Miss Pays de Loire": "Miss Pays de la Loire",
    "Miss Ile-de-France": "Miss Île-de-France",
    "Miss Côte d'Azur": "Miss Côte d'Azur",
    "Miss Centre-Val de Loire": "Miss Centre-Val-de-Loire",
    "Miss Languedoc": "Miss Languedoc-Roussillon",
    "Miss Midi-Pyrénées": "Miss Midi-Pyrénées",
    "Miss Poitou-Charentes": "Miss Poitou-Charentes",
    "Miss Nord-Pas-de-Calais": "Miss Nord-Pas-de-Calais",
    "Miss Franche-Comté": "Miss Franche-Comté",
    "Miss Saint-Martin et Saint-Barthélemy": "Miss Saint-Martin et Saint-Barthélemy",
    "Miss Guadeloupe": "Miss Guadeloupe",
    "Miss Lorraine": "Miss Lorraine",
    "Miss Bourgogne": "Miss Bourgogne",
    "Miss Tahiti": "Miss Tahiti",
    "Miss Provence": "Miss Provence",
    "Miss Guyane": "Miss Guyane",
    "Miss Alsace": "Miss Alsace",
    "Miss Roussillon": "Miss Roussillon",
    "Miss Réunion": "Miss Réunion",
    "Miss Martinique": "Miss Martinique",
    "Miss Corse": "Miss Corse",
    "Miss Champagne-Ardenne": "Miss Champagne-Ardenne",
    "Miss Aquitaine": "Miss Aquitaine",
    "Miss Auvergne": "Miss Auvergne",
    "Miss Bretagne": "Miss Bretagne",
    "Miss Limousin": "Miss Limousin",
    "Miss Picardie": "Miss Picardie",
    "Miss Orléanais": "Miss Orléanais",
    "Miss Nouvelle-Calédonie": "Miss Nouvelle-Calédonie",
    "Miss Saint-Pierre-et-Miquelon": "Miss Saint-Pierre-et-Miquelon",
    "Miss Wallis et Futuna": "Miss Wallis et Futuna",
    "Miss Rhône-Alpes": "Miss Rhône-Alpes",
    "Miss Languedoc-Roussillon": "Miss Languedoc-Roussillon",
    "Miss Normandie": "Miss Normandie"
}

In [105]:
# Appliquer le mapping à df_miss
df_miss['Région'] = df_miss['Région'].replace(mapping)

# Appliquer le mapping aux résultats
for year, df_results in results.items():
    if 'Miss' in df_results.columns:
        df_results['Miss'] = df_results['Miss'].replace(mapping)

# Vérifier les correspondances après remapping
print("Valeurs uniques harmonisées dans df_miss:")
print(df_miss['Région'].unique())

print("\nValeurs uniques harmonisées dans les résultats:")
for year, df_results in results.items():
    if 'Miss' in df_results.columns:
        print(f"Année {year}:")
        print(df_results['Miss'].unique())


Valeurs uniques harmonisées dans df_miss:
['Miss Alsace' 'Miss Aquitaine' 'Miss Auvergne' 'Miss Bourgogne'
 'Miss Bretagne' 'Miss Centre' 'Miss Champagne-Ardenne' 'Miss Corse'
 "Miss Côte d'Azur" 'Miss Franche-Comté' 'Miss Guadeloupe' 'Miss Guyane'
 'Miss Île-de-France' 'Miss Languedoc-Roussillon' 'Miss Limousin'
 'Miss Lorraine' 'Miss Martinique' 'Miss Mayotte' 'Miss Midi-Pyrénées'
 'Miss Nord-Pas-de-Calais' 'Miss Normandie' 'Miss Nouvelle-Calédonie'
 'Miss Orléanais' 'Miss Pays de la Loire' 'Miss Pays de Savoie'
 'Miss Picardie' 'Miss Poitou-Charentes' 'Miss Provence' 'Miss Réunion'
 'Miss Rhône-Alpes' 'Miss Roussillon' 'Miss Saint-Pierre-et-Miquelon'
 'Miss Tahiti' 'Miss Saint-Martin et Saint-Barthélemy'
 'Miss Centre-Val-de-Loire' 'Miss Wallis et Futuna']

Valeurs uniques harmonisées dans les résultats:
Année 2011:
['Miss Bretagne' 'Miss Auvergne' 'Miss Picardie'
 'Miss Languedoc-Roussillon' 'Miss Île-de-France' 'Miss Provence'
 'Miss Martinique' 'Miss Orléanais' 'Miss Roussillon' 

In [106]:
for year, df_results in results.items():
    if 'Miss' in df_results.columns:
        df_results['Miss'] = df_results['Miss'].replace(mapping)

# Vérifier les valeurs uniques après harmonisation
print("Valeurs harmonisées de 'Miss' dans les résultats :")
for year, df_results in results.items():
    if 'Miss' in df_results.columns:
        print(f"Année {year} :")
        print(df_results['Miss'].unique())

Valeurs harmonisées de 'Miss' dans les résultats :
Année 2011 :
['Miss Bretagne' 'Miss Auvergne' 'Miss Picardie'
 'Miss Languedoc-Roussillon' 'Miss Île-de-France' 'Miss Provence'
 'Miss Martinique' 'Miss Orléanais' 'Miss Roussillon' 'Miss Corse'
 'Miss Normandie' "Miss Côte d'Azur"]
Année 2012 :
['Miss Réunion' 'Miss Alsace' "Miss Côte d'Azur" 'Miss Pays de la Loire'
 'Miss Provence' 'Miss Martinique' 'Miss Languedoc-Roussillon'
 'Miss Guyane' 'Miss Bretagne' 'Miss Roussillon' 'Miss Bourgogne'
 'Miss Midi-Pyrénées']
Année 2013 :
['Miss Bourgogne' 'Miss Tahiti' 'Miss Nord-Pas-de-Calais'
 'Miss Martinique' 'Miss Pays de la Loire' 'Miss Rhône-Alpes'
 "Miss Côte d'Azur" 'Miss Champagne-Ardenne' 'Miss Provence'
 'Miss Picardie' 'Miss Bretagne' 'Miss Roussillon']
Année 2014 :
['Miss Tahiti' 'Miss Orléanais' "Miss Côte d'Azur" 'Miss Provence'
 'Miss Guadeloupe' 'Miss Auvergne' 'Miss Languedoc-Roussillon'
 'Miss Alsace' 'Miss Franche-Comté' 'Miss Roussillon' 'Miss Bourgogne'
 'Miss Réunion']
A

In [107]:
df_miss['année'] = pd.to_numeric(df_miss['année'], errors='coerce')

# Vérifier si des valeurs NaN existent après la conversion
if df_miss['année'].isna().any():
    print("Certaines valeurs dans 'année' ne sont pas valides et ont été converties en NaN.")
    print(df_miss[df_miss['année'].isna()])

# Remplacer les valeurs NaN (si nécessaire)
df_miss['année'] = df_miss['année'].fillna(0).astype(int)

In [108]:
# Mettre à jour df_miss avec les données des résultats
for year, df_results in results.items():
    if 'Miss' in df_results.columns:
        for _, row in df_results.iterrows():
            condition_1 = (df_miss['Région'] == row['Miss'])
            condition_2 = (df_miss['année'] == int(year))
            condition= condition_1 & condition_2
            if condition.any():
                print('Condition trouvée')
                df_miss.loc[condition, 'Top 15/12'] = True
                df_miss.loc[condition, 'Points_du_public'] = row['Public']
                df_miss.loc[condition, 'Points_du_jury'] = row['Jury']
                df_miss.loc[condition, 'Points_totaux'] = row['Total']
            else :
                print('pas de miss trouvée')

# Afficher les résultats taggés
print("\nRésultats taggés (Top 15/12):")
print(df_miss[df_miss['Top 15/12'] == True][['Nom', 'année', 'Top 15/12', 'Points_du_public', 'Points_du_jury', 'Points_totaux']])

Condition trouvée
Condition trouvée
Condition trouvée
Condition trouvée
Condition trouvée
Condition trouvée
Condition trouvée
Condition trouvée
Condition trouvée
Condition trouvée
Condition trouvée
Condition trouvée
Condition trouvée
Condition trouvée
Condition trouvée
Condition trouvée
Condition trouvée
Condition trouvée
Condition trouvée
Condition trouvée
Condition trouvée
Condition trouvée
Condition trouvée
Condition trouvée
Condition trouvée
Condition trouvée
Condition trouvée
Condition trouvée
Condition trouvée
Condition trouvée
Condition trouvée
Condition trouvée
Condition trouvée
Condition trouvée
Condition trouvée
Condition trouvée
Condition trouvée
Condition trouvée
Condition trouvée
Condition trouvée
Condition trouvée
Condition trouvée
Condition trouvée
Condition trouvée
Condition trouvée
Condition trouvée
Condition trouvée
Condition trouvée
Condition trouvée
Condition trouvée
Condition trouvée
Condition trouvée
Condition trouvée
Condition trouvée
Condition trouvée
Condition 

C:\Users\charl\AppData\Local\Temp\ipykernel_42008\2123396742.py:11: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '12' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_miss.loc[condition, 'Points_du_public'] = row['Public']
C:\Users\charl\AppData\Local\Temp\ipykernel_42008\2123396742.py:12: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '11' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_miss.loc[condition, 'Points_du_jury'] = row['Jury']
C:\Users\charl\AppData\Local\Temp\ipykernel_42008\2123396742.py:13: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '23' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_miss.loc[cond

Condition trouvée
Condition trouvée
Condition trouvée
Condition trouvée
Condition trouvée
Condition trouvée
Condition trouvée
Condition trouvée
Condition trouvée
Condition trouvée
Condition trouvée
Condition trouvée
Condition trouvée
Condition trouvée
Condition trouvée
Condition trouvée
Condition trouvée
Condition trouvée
Condition trouvée
Condition trouvée
Condition trouvée
Condition trouvée
Condition trouvée
Condition trouvée
Condition trouvée
Condition trouvée
Condition trouvée
Condition trouvée
Condition trouvée
Condition trouvée
Condition trouvée
Condition trouvée
Condition trouvée
Condition trouvée
Condition trouvée
Condition trouvée
Condition trouvée
Condition trouvée
Condition trouvée
Condition trouvée
Condition trouvée
Condition trouvée
Condition trouvée
Condition trouvée
Condition trouvée
Condition trouvée
Condition trouvée
Condition trouvée
Condition trouvée
Condition trouvée
Condition trouvée
Condition trouvée
Condition trouvée
Condition trouvée
Condition trouvée
Condition 

# Get data top 5

In [109]:
sections = ['Résultats du Top 5', 'Troisième tour', 'Dernier tour']

# Dictionnaire pour stocker les résultats
top5_results = {}

# Parcourir chaque URL
for url in url_list:
    year = url.split('_')[-1]  # Extraire l'année de l'URL
    print(f"Scraping pour l'année {year}...")

    # Charger le contenu de la page
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    # Rechercher les sections pertinentes
    table_found = False
    for section_title in sections:
        header = None
        for header_candidate in soup.find_all(['h1', 'h2', 'h3', 'h4', 'h5', 'h6']):
            if section_title in header_candidate.text:
                header = header_candidate
                break

        # Si la section est trouvée, chercher un tableau
        if header:
            table = header.find_next('table', {'class': 'wikitable'})
            print(table)
            if table:
                headers = [th.text.strip() for th in table.find_all('th')]
                rows = []
                for row in table.find_all('tr')[1:]:
                    cols = [td.text.strip() for td in row.find_all('td')]
                    rows.append(cols)
                
                print(f"Headers ({len(headers)}): {headers}")
                print(f"First row ({len(rows[0])}): {rows[0] if rows else 'No rows found'}")

                # Vérifier si le nombre de colonnes est cohérent
                if rows and len(headers) != len(rows[0]):
                    print(f"Incohérence détectée : {len(headers)} colonnes dans les headers, mais {len(rows[0])} colonnes dans la première ligne.")
                    # Ajuster automatiquement les colonnes manquantes ou en trop
                    max_columns = max(len(headers), len(rows[0]))
                    headers = headers + [''] * (max_columns - len(headers))  # Compléter les headers
                    rows = [row + [''] * (max_columns - len(row)) for row in rows]  # Compléter les lignes
                
                # Convertir en DataFrame
                if rows:
                    df = pd.DataFrame(rows, columns=headers)
                    top5_results[year] = df
                    table_found = True
                    print(f"Tableau trouvé pour l'année {year} dans la section '{section_title}'")
                    break  # Passer à l'URL suivante si un tableau est trouvé

        if table_found:
            break  # Passer à l'URL suivante si un tableau est trouvé

    if not table_found:
        print(f"Aucun tableau trouvé pour l'année {year}.")

# Vérifier les résultats
for year, df in top5_results.items():
    print(f"\nRésultats du Top 5 pour l'année {year} :")
    print(df.head())

Scraping pour l'année 2011...
<table class="wikitable" style="text-align:center;">
<tbody><tr>
<th scope="col">Candidate
</th>
<th scope="col">Résultat
</th></tr>
<tr>
<th><a href="/wiki/Miss_Bretagne" title="Miss Bretagne">Miss Bretagne</a>
</th>
<td>37,90 %
</td></tr>
<tr>
<th><a class="mw-redirect" href="/wiki/Miss_Languedoc" title="Miss Languedoc">Miss Languedoc</a>
</th>
<td>17,80 %
</td></tr>
<tr>
<th><a href="/wiki/Miss_Auvergne" title="Miss Auvergne">Miss Auvergne</a>
</th>
<td>16,30 %
</td></tr>
<tr>
<th><a href="/wiki/Miss_Picardie" title="Miss Picardie">Miss Picardie</a>
</th>
<td>14,90 %
</td></tr>
<tr>
<th><a href="/wiki/Miss_%C3%8Ele-de-France" title="Miss Île-de-France">Miss Île-de-France</a>
</th>
<td>13,10 %
</td></tr></tbody></table>
Headers (7): ['Candidate', 'Résultat', 'Miss Bretagne', 'Miss Languedoc', 'Miss Auvergne', 'Miss Picardie', 'Miss Île-de-France']
First row (1): ['37,90\xa0%']
Incohérence détectée : 7 colonnes dans les headers, mais 1 colonnes dans la pr

In [110]:
top5_results

{'2011':   Candidate Résultat Miss Bretagne Miss Languedoc Miss Auvergne Miss Picardie  \
 0   37,90 %                                                                     
 1   17,80 %                                                                     
 2   16,30 %                                                                     
 3   14,90 %                                                                     
 4   13,10 %                                                                     
 
   Miss Île-de-France  
 0                     
 1                     
 2                     
 3                     
 4                     ,
 '2012':   Candidate Résultat Miss Alsace Miss Pays de Loire Miss Réunion  \
 0    32,3 %                                                        
 1    31,6 %                                                        
 2    23,5 %                                                        
 3     6,6 %                                                        


In [111]:

url=url_list[11]
print(url)
year = url.split('_')[-1]  # Extraire l'année de l'URL
print(f"Scraping pour l'année {year}...")

# Charger le contenu de la page
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

header = None
for header_candidate in soup.find_all(['h1', 'h2', 'h3', 'h4', 'h5', 'h6']):
    if "Second tour" in header_candidate.text:
        header = header_candidate
        break
if header:
    table = header.find_next('table', {'class': 'wikitable'})
    print(table)
    if table:
        headers = [th.text.strip() for th in table.find_all('th')]
        rows = []
        for row in table.find_all('tr')[1:]:
            cols = [td.text.strip() for td in row.find_all('td')]
            rows.append(cols)
        
        print(f"Headers ({len(headers)}): {headers}")
        print(f"First row ({len(rows[0])}): {rows[0] if rows else 'No rows found'}")

        # Vérifier si le nombre de colonnes est cohérent
        if rows and len(headers) != len(rows[0]):
            print(f"Incohérence détectée : {len(headers)} colonnes dans les headers, mais {len(rows[0])} colonnes dans la première ligne.")
            # Ajuster automatiquement les colonnes manquantes ou en trop
            max_columns = max(len(headers), len(rows[0]))
            headers = headers + [''] * (max_columns - len(headers))  # Compléter les headers
            rows = [row + [''] * (max_columns - len(row)) for row in rows]  # Compléter les lignes
        
        # Convertir en DataFrame
        if rows:
            df = pd.DataFrame(rows, columns=headers)
            top5_results[year] = df
            table_found = True
            print(f"Tableau trouvé pour l'année {year} dans la section '{section_title}'")
            
else:
    print(f"Aucun tableau trouvé pour l'année {year} dans 'Second tour'.")


# Vérifier les résultats
for year, df in top5_results.items():
    print(f"\nRésultats pour l'année {year} :")
    print(df.head())

https://fr.wikipedia.org/wiki/Miss_France_2022
Scraping pour l'année 2022...
<table class="wikitable sortable" style="text-align:center;">
<tbody><tr>
<th scope="col">N°
</th>
<th scope="col">Candidates
</th>
<th scope="col">Public
</th>
<th scope="col">Jury
</th>
<th scope="col">Total
</th></tr>
<tr>
<th scope="row">1
</th>
<td bgcolor="gold">Miss Île-de-France
</td>
<td>3
</td>
<td>5
</td>
<td>8
</td></tr>
<tr>
<th scope="row">2
</th>
<td bgcolor="silver">Miss Martinique
</td>
<td>5
</td>
<td>1
</td>
<td>6
</td></tr>
<tr>
<th scope="row">3
</th>
<td bgcolor="#cc9966">Miss Alsace
</td>
<td>4
</td>
<td>2
</td>
<td>6
</td></tr>
<tr>
<th scope="row">4
</th>
<td>Miss Tahiti
</td>
<td>2
</td>
<td>4
</td>
<td>6
</td></tr>
<tr>
<th scope="row">5
</th>
<td>Miss Normandie
</td>
<td>1
</td>
<td>4
</td>
<td>5
</td></tr>
</tbody></table>
Headers (10): ['N°', 'Candidates', 'Public', 'Jury', 'Total', '1', '2', '3', '4', '5']
First row (4): ['Miss Île-de-France', '3', '5', '8']
Incohérence détectée 

In [112]:
# URL de la page pour Miss France 2023
url = 'https://fr.wikipedia.org/wiki/Miss_France_2023'

# Envoyer une requête avec un User-Agent pour éviter les restrictions
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36'
}
response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.content, 'html.parser')

# Trouver la section "Classement"
header = None
for header_candidate in soup.find_all(['h1', 'h2', 'h3', 'h4', 'h5', 'h6']):
    if "Classement" in header_candidate.text:
        header = header_candidate
        break

# Extraire le deuxième tableau dans cette section
if header:
    tables = header.find_all_next('table', {'class': 'wikitable'})
    if len(tables) > 1:  # Vérifier qu'il y a un second tableau
        table = tables[5]  # Prendre le deuxième tableau
        print(table)
        headers = [th.text.strip() for th in table.find_all('th')]
        rows = []
        for row in table.find_all('tr')[1:]:
            cols = [td.text.strip() for td in row.find_all('td')]
            rows.append(cols)

        print(f"Headers ({len(headers)}): {headers}")
        print(f"First row ({len(rows[0]) if rows else 0}): {rows[0] if rows else 'No rows found'}")

        # Vérifier si le nombre de colonnes est cohérent
        if rows and len(headers) != len(rows[0]):
            print(f"Incohérence détectée : {len(headers)} colonnes dans les headers, mais {len(rows[0])} colonnes dans la première ligne.")
            max_columns = max(len(headers), len(rows[0]))
            headers = headers + [''] * (max_columns - len(headers))  # Compléter les headers
            rows = [row + [''] * (max_columns - len(row)) for row in rows]  # Compléter les lignes

        # Convertir en DataFrame
        if rows:
            df_results_2023 = pd.DataFrame(rows, columns=headers)
            print("Classement des candidates pour 2023 :")
            print(df_results_2023)
    else:
        print("Aucun second tableau trouvé dans la section 'Classement' pour 2023.")
else:
    print("Section 'Classement' introuvable pour l'année 2023.")

<table class="wikitable sortable" style="text-align:center;">
<tbody><tr>
<th scope="col">N°
</th>
<th scope="col">Candidates
</th>
<th scope="col">Public
</th>
<th scope="col">Jury
</th>
<th scope="col">Total
</th></tr>
<tr>
<th scope="row">1
</th>
<td bgcolor="gold">Miss Guadeloupe
</td>
<td>5
</td>
<td>5
</td>
<td>10
</td></tr>
<tr>
<th scope="row">2
</th>
<td bgcolor="silver">Miss Nord-Pas-de-Calais
</td>
<td>4
</td>
<td>4
</td>
<td>8
</td></tr>
<tr>
<th scope="row">3
</th>
<td bgcolor="#cc9966">Miss Franche-Comté
</td>
<td>2
</td>
<td>3
</td>
<td>5
</td></tr>
<tr>
<th scope="row">4
</th>
<td>Miss Martinique
</td>
<td>3
</td>
<td>1
</td>
<td>4
</td></tr>
<tr>
<th scope="row">5
</th>
<td>Miss Auvergne
</td>
<td>1
</td>
<td>2
</td>
<td>3
</td></tr></tbody></table>
Headers (10): ['N°', 'Candidates', 'Public', 'Jury', 'Total', '1', '2', '3', '4', '5']
First row (4): ['Miss Guadeloupe', '5', '5', '10']
Incohérence détectée : 10 colonnes dans les headers, mais 4 colonnes dans la première

In [113]:
df_results_2023

,N°,Candidates,Public,Jury,Total,1,2,3,4,5
0,Miss Guadeloupe,5,5,10,,,,,,
1,Miss Nord-Pas-de-Calais,4,4,8,,,,,,
2,Miss Franche-Comté,2,3,5,,,,,,
3,Miss Martinique,3,1,4,,,,,,
4,Miss Auvergne,1,2,3,,,,,,


In [114]:
top5_results['2023']=df_results_2023

In [115]:
top5_results

{'2011':   Candidate Résultat Miss Bretagne Miss Languedoc Miss Auvergne Miss Picardie  \
 0   37,90 %                                                                     
 1   17,80 %                                                                     
 2   16,30 %                                                                     
 3   14,90 %                                                                     
 4   13,10 %                                                                     
 
   Miss Île-de-France  
 0                     
 1                     
 2                     
 3                     
 4                     ,
 '2012':   Candidate Résultat Miss Alsace Miss Pays de Loire Miss Réunion  \
 0    32,3 %                                                        
 1    31,6 %                                                        
 2    23,5 %                                                        
 3     6,6 %                                                        


# Tag top 5

In [116]:
for year, df in top5_results.items() :
    df['Classement'] = df_results_2023.index + 1

# Afficher le DataFrame mis à jour
    print("Classement des candidates avec colonne Classement ajoutée :")
    print(df)

Classement des candidates avec colonne Classement ajoutée :
  Candidate Résultat Miss Bretagne Miss Languedoc Miss Auvergne Miss Picardie  \
0   37,90 %                                                                     
1   17,80 %                                                                     
2   16,30 %                                                                     
3   14,90 %                                                                     
4   13,10 %                                                                     

  Miss Île-de-France  Classement  
0                              1  
1                              2  
2                              3  
3                              4  
4                              5  
Classement des candidates avec colonne Classement ajoutée :
  Candidate Résultat Miss Alsace Miss Pays de Loire Miss Réunion  \
0    32,3 %                                                        
1    31,6 %                                    

In [117]:
print(top5_results['2011'])

  Candidate Résultat Miss Bretagne Miss Languedoc Miss Auvergne Miss Picardie  \
0   37,90 %                                                                     
1   17,80 %                                                                     
2   16,30 %                                                                     
3   14,90 %                                                                     
4   13,10 %                                                                     

  Miss Île-de-France  Classement  
0                              1  
1                              2  
2                              3  
3                              4  
4                              5  


## On corrige les dataframes

In [118]:
# Parcourir les DataFrames dans top5_results
for year, df in top5_results.items():
    print(f"Nettoyage des colonnes pour l'année {year}...")
    
    # Identifier les colonnes contenant "Résultat" (avec ou sans typo)
    resultat_columns = [col for col in df.columns if "Résultat" in col]

    # Renommer les colonnes détectées en "Résultat"
    df.rename(columns={col: "Résultat" for col in resultat_columns}, inplace=True)
    
    # Mettre à jour dans le dictionnaire
    top5_results[year] = df

    print(f"Colonnes après nettoyage : {list(df.columns)}")

# Vérifier un exemple
print("\nExemple de DataFrame nettoyé :")
print(top5_results[list(top5_results.keys())[0]].head())

Nettoyage des colonnes pour l'année 2011...
Colonnes après nettoyage : ['Candidate', 'Résultat', 'Miss Bretagne', 'Miss Languedoc', 'Miss Auvergne', 'Miss Picardie', 'Miss Île-de-France', 'Classement']
Nettoyage des colonnes pour l'année 2012...
Colonnes après nettoyage : ['Candidate', 'Résultat', 'Miss Alsace', 'Miss Pays de Loire', 'Miss Réunion', 'Miss Provence', "Miss Côte d'Azur", 'Classement']
Nettoyage des colonnes pour l'année 2013...
Colonnes après nettoyage : ['Candidate', 'Résultat', 'Miss Bourgogne', 'Miss Tahiti', 'Miss Nord-Pas-de-Calais', 'Miss Martinique', 'Miss Pays de Loire', 'Classement']
Nettoyage des colonnes pour l'année 2014...
Colonnes après nettoyage : ['Candidates', 'Résultat', 'Miss Orléanais', 'Miss Tahiti', 'Miss Provence', "Miss Côte d'Azur", 'Miss Guadeloupe', 'Classement']
Nettoyage des colonnes pour l'année 2015...
Colonnes après nettoyage : ['Candidates', 'Résultat', 'Miss Nord-Pas-de-Calais', 'Miss Tahiti', "Miss Côte d'Azur", 'Miss Aquitaine', 'Miss 

In [119]:
for year, df in top5_results.items():
    df = top5_results[year]

    miss_columns = [col for col in df.columns if col.startswith('Miss')]

    if len(miss_columns)>0:
        # Print the ordered list
        print("Ordered list of columns starting with 'Miss':")
        print(miss_columns)

        # Drop 'Résultat' and the columns starting with 'Miss'
        df = df.drop(columns=miss_columns + ['Résultat'])

        # Rename the column 'Candidates' to 'Résultat'
        df = df.rename(columns={'Candidates': 'Résultat'})

        df['Candidates'] = miss_columns[:len(df)]  # Ensure alignment with the number of rows

        # Display the transformed DataFrame
        print(f"Transformed DataFrame yor year {year}:")
        print(df)

        top5_results[year]=df
    else :
        print(f'No need to transform dataframe for year {year}')
        print(df)

Ordered list of columns starting with 'Miss':
['Miss Bretagne', 'Miss Languedoc', 'Miss Auvergne', 'Miss Picardie', 'Miss Île-de-France']
Transformed DataFrame yor year 2011:
  Candidate  Classement          Candidates
0   37,90 %           1       Miss Bretagne
1   17,80 %           2      Miss Languedoc
2   16,30 %           3       Miss Auvergne
3   14,90 %           4       Miss Picardie
4   13,10 %           5  Miss Île-de-France
Ordered list of columns starting with 'Miss':
['Miss Alsace', 'Miss Pays de Loire', 'Miss Réunion', 'Miss Provence', "Miss Côte d'Azur"]
Transformed DataFrame yor year 2012:
  Candidate  Classement          Candidates
0    32,3 %           1         Miss Alsace
1    31,6 %           2  Miss Pays de Loire
2    23,5 %           3        Miss Réunion
3     6,6 %           4       Miss Provence
4       6 %           5    Miss Côte d'Azur
Ordered list of columns starting with 'Miss':
['Miss Bourgogne', 'Miss Tahiti', 'Miss Nord-Pas-de-Calais', 'Miss Martinique

In [120]:
top5_results['2018']

,Résultat,Classement,Candidates
0,"29,20 %[60]",1,Miss Nord-Pas-de-Calais
1,"25,26 %[60]",2,Miss Corse
2,"18,18 %[60]",3,Miss Île-de-France
3,"15,17 %[60]",4,Miss Champagne-Ardenne
4,"12,19 %[60]",5,Miss Réunion


Mais pas même format avec 2021 à 2024

In [121]:
top5_results['2022']

,N°,Candidates,Public,Jury,Total,1,2,3,4,5,Classement
0,Miss Île-de-France,3,5,8,,,,,,,1
1,Miss Martinique,5,1,6,,,,,,,2
2,Miss Alsace,4,2,6,,,,,,,3
3,Miss Tahiti,2,4,6,,,,,,,4
4,Miss Normandie,1,4,5,,,,,,,5


In [122]:
def clean_and_rename_columns(df):
    columns_to_drop = ['1', '2', '3', '4', '5', 'Total']
    df = df.drop(columns=[col for col in columns_to_drop if col in df.columns])
    
    column_mapping = {
        'Jury': 'Total',
        'Public': 'Jury',
        'Candidates': 'Public',
        'N°': 'Candidates'
    }
    df = df.rename(columns=column_mapping)
    
    return df

In [123]:
for year, df in top5_results.items():
    if int(year) >=2021 :
        print(f"Nettoyage des colonnes pour l'année {year}...")
        top5_results[year] = clean_and_rename_columns(df)

Nettoyage des colonnes pour l'année 2021...
Nettoyage des colonnes pour l'année 2024...
Nettoyage des colonnes pour l'année 2022...
Nettoyage des colonnes pour l'année 2023...


In [124]:
top5_results['2022']

,Candidates,Public,Jury,Total,Classement
0,Miss Île-de-France,3,5,8,1
1,Miss Martinique,5,1,6,2
2,Miss Alsace,4,2,6,3
3,Miss Tahiti,2,4,6,4
4,Miss Normandie,1,4,5,5


## Harmonisation Candidates

In [125]:
def harmonize_candidates(df, mapping):
    if 'Candidates' in df.columns:
        df['Candidates'] = df['Candidates'].replace(mapping)
    return df

In [126]:
for year, df in top5_results.items():
    print(f"Harmonisation des candidates pour l'année {year}...")
    top5_results[year] = harmonize_candidates(df, mapping)
    print(f"Exemple après harmonisation pour {year} :")
    print(top5_results[year].head())

print("\nExemple harmonisé pour une année (2023 si existante) :")
if '2023' in top5_results:
    print(top5_results['2023'].head())
else:
    print("2023 n'est pas dans les résultats.")

Harmonisation des candidates pour l'année 2011...
Exemple après harmonisation pour 2011 :
  Candidate  Classement                 Candidates
0   37,90 %           1              Miss Bretagne
1   17,80 %           2  Miss Languedoc-Roussillon
2   16,30 %           3              Miss Auvergne
3   14,90 %           4              Miss Picardie
4   13,10 %           5         Miss Île-de-France
Harmonisation des candidates pour l'année 2012...
Exemple après harmonisation pour 2012 :
  Candidate  Classement             Candidates
0    32,3 %           1            Miss Alsace
1    31,6 %           2  Miss Pays de la Loire
2    23,5 %           3           Miss Réunion
3     6,6 %           4          Miss Provence
4       6 %           5       Miss Côte d'Azur
Harmonisation des candidates pour l'année 2013...
Exemple après harmonisation pour 2013 :
                  Candidate  Classement               Candidates
0               41,67 %[10]           1           Miss Bourgogne
1           

In [127]:
# Valider les valeurs harmonisées
harmonized_names = set(mapping.values())
for year, df in top5_results.items():
    if 'Candidates' in df.columns:
        invalid_names = set(df['Candidates']) - harmonized_names
        if invalid_names:
            print(f"Noms non harmonisés trouvés pour l'année {year}: {invalid_names}")

## Correction colonne 'Résultats'

In [128]:
for year, df in top5_results.items():
    df = df.rename(columns={'Candidate': 'Résultat'})
    top5_results[year]=df

In [129]:
top5_results['2012']

,Résultat,Classement,Candidates
0,"32,3 %",1,Miss Alsace
1,"31,6 %",2,Miss Pays de la Loire
2,"23,5 %",3,Miss Réunion
3,"6,6 %",4,Miss Provence
4,6 %,5,Miss Côte d'Azur


In [130]:
def clean_result_column(df):
    if 'Résultat' in df.columns:
        # Supprimer les crochets et leur contenu, ainsi que les espaces inutiles
        df['Résultat'] = df['Résultat'].str.replace(r'\[.*?\]', '', regex=True).str.strip()
    return df

In [131]:
for year, df in top5_results.items():
    print(f"Nettoyage de la colonne 'Résultat' pour l'année {year}...")
    top5_results[year] = clean_result_column(df)

print("\nExemple après nettoyage :")
for year, df in top5_results.items():
    print(f"Année {year} :")
    print(df.head())
    break

Nettoyage de la colonne 'Résultat' pour l'année 2011...
Nettoyage de la colonne 'Résultat' pour l'année 2012...
Nettoyage de la colonne 'Résultat' pour l'année 2013...
Nettoyage de la colonne 'Résultat' pour l'année 2014...
Nettoyage de la colonne 'Résultat' pour l'année 2015...
Nettoyage de la colonne 'Résultat' pour l'année 2016...
Nettoyage de la colonne 'Résultat' pour l'année 2017...
Nettoyage de la colonne 'Résultat' pour l'année 2018...
Nettoyage de la colonne 'Résultat' pour l'année 2019...
Nettoyage de la colonne 'Résultat' pour l'année 2020...
Nettoyage de la colonne 'Résultat' pour l'année 2021...
Nettoyage de la colonne 'Résultat' pour l'année 2024...
Nettoyage de la colonne 'Résultat' pour l'année 2022...
Nettoyage de la colonne 'Résultat' pour l'année 2023...

Exemple après nettoyage :
Année 2011 :
  Résultat  Classement                 Candidates
0  37,90 %           1              Miss Bretagne
1  17,80 %           2  Miss Languedoc-Roussillon
2  16,30 %           3    

## Création colonne + tagger les miss

In [132]:
df_miss['top_5'] = False
df_miss['Classement'] = None

In [133]:
df_miss

,Région,Nom,Âge,Taille,Résidence,Date élection,Classement Final,année,Top 15/12,Points_du_public,Points_du_jury,Points_totaux,top_5,Classement
0,Miss Alsace,Mathilde Buecher,19,1.78,Heimersdorf,2010-10-10,,2011,False,0,0,0,False,None
1,Miss Aquitaine,Clémence Thill,20,1.75,Agen,2010-10-16,,2011,False,0,0,0,False,None
2,Miss Auvergne,Clémence Oleksy,19,1.76,Vichy,2010-10-17,2e dauphine,2011,True,10,12,22,False,None
3,Miss Bourgogne,Alice Detollenaere,23,1.74,Chitry-les-Mines,2017-09-29,Prix du Mannequinat,2011,False,0,0,0,False,None
4,Miss Bretagne,Laury Thilleman,19,1.78,Brest,2010-10-26,Miss France 2011\nPrix de la Culture générale,2011,True,12,11,23,False,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
429,Miss Provence,Adélina Blanc,25,1.73,Eyragues,2023-07-27,2e dauphine,2024,True,12,11,23,False,None
430,Miss Réunion,Mélanie Odules,20,1.78,Saint-Paul,2023-08-26,,2024,False,0,0,0,False,None
431,Miss Rhône-Alpes,Alizée Bidaut,22,1.74,Saint-Genis-sur-Menthon,2023-09-23,,2024,False,0,0,0,False,None
432,Miss Roussillon,Élise Aquilina,21,1.71,Cabestany,2023-08-05,Top 15 (11e)\nPrix de l’académie des Miss[25],2024,True,6,6,12,False,None


In [134]:
for year, df_top5 in top5_results.items():
    print(f"Tagging top 5 pour l'année {year}...")
    
    if 'Candidates' in df_top5.columns and 'Classement' in df_top5.columns:
        for _, row in df_top5.iterrows():
            candidate = row['Candidates']
            classement = row['Classement']
            
            condition = (df_miss['Région'] == candidate) & (df_miss['année'] == int(year))
            if condition.any():
                df_miss.loc[condition, 'top_5'] = True
                df_miss.loc[condition, 'Classement'] = classement

print("\nRésultats taggés (top_5):")
print(df_miss[df_miss['top_5']].head())

Tagging top 5 pour l'année 2011...
Tagging top 5 pour l'année 2012...
Tagging top 5 pour l'année 2013...
Tagging top 5 pour l'année 2014...
Tagging top 5 pour l'année 2015...
Tagging top 5 pour l'année 2016...
Tagging top 5 pour l'année 2017...
Tagging top 5 pour l'année 2018...
Tagging top 5 pour l'année 2019...
Tagging top 5 pour l'année 2020...
Tagging top 5 pour l'année 2021...
Tagging top 5 pour l'année 2024...
Tagging top 5 pour l'année 2022...
Tagging top 5 pour l'année 2023...

Résultats taggés (top_5):
                       Région                   Nom  Âge  Taille  \
2               Miss Auvergne       Clémence Oleksy   19    1.76   
4               Miss Bretagne       Laury Thilleman   19    1.78   
12         Miss Île-de-France   Sabine Hossenbaccus   22    1.70   
13  Miss Languedoc-Roussillon       Jenna Sylvestre   22    1.72   
25              Miss Picardie  Anastasia Winnebroot   20    1.77   

          Résidence Date élection  \
2             Vichy    2010-10-17   


In [135]:
df_miss

,Région,Nom,Âge,Taille,Résidence,Date élection,Classement Final,année,Top 15/12,Points_du_public,Points_du_jury,Points_totaux,top_5,Classement
0,Miss Alsace,Mathilde Buecher,19,1.78,Heimersdorf,2010-10-10,,2011,False,0,0,0,False,None
1,Miss Aquitaine,Clémence Thill,20,1.75,Agen,2010-10-16,,2011,False,0,0,0,False,None
2,Miss Auvergne,Clémence Oleksy,19,1.76,Vichy,2010-10-17,2e dauphine,2011,True,10,12,22,True,3
3,Miss Bourgogne,Alice Detollenaere,23,1.74,Chitry-les-Mines,2017-09-29,Prix du Mannequinat,2011,False,0,0,0,False,None
4,Miss Bretagne,Laury Thilleman,19,1.78,Brest,2010-10-26,Miss France 2011\nPrix de la Culture générale,2011,True,12,11,23,True,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
429,Miss Provence,Adélina Blanc,25,1.73,Eyragues,2023-07-27,2e dauphine,2024,True,12,11,23,True,3
430,Miss Réunion,Mélanie Odules,20,1.78,Saint-Paul,2023-08-26,,2024,False,0,0,0,False,None
431,Miss Rhône-Alpes,Alizée Bidaut,22,1.74,Saint-Genis-sur-Menthon,2023-09-23,,2024,False,0,0,0,False,None
432,Miss Roussillon,Élise Aquilina,21,1.71,Cabestany,2023-08-05,Top 15 (11e)\nPrix de l’académie des Miss[25],2024,True,6,6,12,False,None


# Tag top 6 -12 to 15

In [136]:
for year in years:
    print(f"Traitement pour l'année {year}...")
    df_year = df_miss[df_miss['année'] == year]
    
    top_6_12_15 = df_year[(df_year['Top 15/12']) & (~df_year['top_5'])]
    
    top_6_12_15 = top_6_12_15.sort_values(by='Points_totaux', ascending=False)
    
    # Ajouter un classement (6 à 12 ou 15 selon le nombre d'entrées)
    for rank, index in enumerate(top_6_12_15.index, start=6):
        df_miss.loc[index, 'Classement'] = rank

# Afficher les résultats
print("\nMiss taggées pour le classement 6-12/15 :")
print(df_miss[df_miss['Classement'].notnull()].sort_values(['année', 'Classement']).head())


Traitement pour l'année année 2011...
Traitement pour l'année année 2012...
Traitement pour l'année année 2013...
Traitement pour l'année année 2014...
Traitement pour l'année année 2015...
Traitement pour l'année année 2016...
Traitement pour l'année année 2017...
Traitement pour l'année année 2018...
Traitement pour l'année année 2019...
Traitement pour l'année année 2020...
Traitement pour l'année année 2021...
Traitement pour l'année année 2022...
Traitement pour l'année année 2023...
Traitement pour l'année année 2024...

Miss taggées pour le classement 6-12/15 :
                       Région                   Nom  Âge  Taille  \
4               Miss Bretagne       Laury Thilleman   19    1.78   
13  Miss Languedoc-Roussillon       Jenna Sylvestre   22    1.72   
2               Miss Auvergne       Clémence Oleksy   19    1.76   
25              Miss Picardie  Anastasia Winnebroot   20    1.77   
12         Miss Île-de-France   Sabine Hossenbaccus   22    1.70   

          Réside

# Encodage des régions

In [137]:
unique_regions = df_miss['Région'].unique()
region_mapping = {region: idx + 1 for idx, region in enumerate(unique_regions)}

df_miss['ID_REGION'] = df_miss['Région'].map(region_mapping)

print("Mapping des régions vers ID_REGION :")
print(region_mapping)

print("\nAperçu de df_miss avec la colonne ID_REGION :")
print(df_miss[['Région', 'ID_REGION']].head())

Mapping des régions vers ID_REGION :
{'Miss Alsace': 1, 'Miss Aquitaine': 2, 'Miss Auvergne': 3, 'Miss Bourgogne': 4, 'Miss Bretagne': 5, 'Miss Centre': 6, 'Miss Champagne-Ardenne': 7, 'Miss Corse': 8, "Miss Côte d'Azur": 9, 'Miss Franche-Comté': 10, 'Miss Guadeloupe': 11, 'Miss Guyane': 12, 'Miss Île-de-France': 13, 'Miss Languedoc-Roussillon': 14, 'Miss Limousin': 15, 'Miss Lorraine': 16, 'Miss Martinique': 17, 'Miss Mayotte': 18, 'Miss Midi-Pyrénées': 19, 'Miss Nord-Pas-de-Calais': 20, 'Miss Normandie': 21, 'Miss Nouvelle-Calédonie': 22, 'Miss Orléanais': 23, 'Miss Pays de la Loire': 24, 'Miss Pays de Savoie': 25, 'Miss Picardie': 26, 'Miss Poitou-Charentes': 27, 'Miss Provence': 28, 'Miss Réunion': 29, 'Miss Rhône-Alpes': 30, 'Miss Roussillon': 31, 'Miss Saint-Pierre-et-Miquelon': 32, 'Miss Tahiti': 33, 'Miss Saint-Martin et Saint-Barthélemy': 34, 'Miss Centre-Val-de-Loire': 35, 'Miss Wallis et Futuna': 36}

Aperçu de df_miss avec la colonne ID_REGION :
           Région  ID_REGION

In [138]:
df_miss

,Région,Nom,Âge,Taille,Résidence,Date élection,Classement Final,année,Top 15/12,Points_du_public,Points_du_jury,Points_totaux,top_5,Classement,ID_REGION
0,Miss Alsace,Mathilde Buecher,19,1.78,Heimersdorf,2010-10-10,,2011,False,0,0,0,False,None,1
1,Miss Aquitaine,Clémence Thill,20,1.75,Agen,2010-10-16,,2011,False,0,0,0,False,None,2
2,Miss Auvergne,Clémence Oleksy,19,1.76,Vichy,2010-10-17,2e dauphine,2011,True,10,12,22,True,3,3
3,Miss Bourgogne,Alice Detollenaere,23,1.74,Chitry-les-Mines,2017-09-29,Prix du Mannequinat,2011,False,0,0,0,False,None,4
4,Miss Bretagne,Laury Thilleman,19,1.78,Brest,2010-10-26,Miss France 2011\nPrix de la Culture générale,2011,True,12,11,23,True,1,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
429,Miss Provence,Adélina Blanc,25,1.73,Eyragues,2023-07-27,2e dauphine,2024,True,12,11,23,True,3,28
430,Miss Réunion,Mélanie Odules,20,1.78,Saint-Paul,2023-08-26,,2024,False,0,0,0,False,None,29
431,Miss Rhône-Alpes,Alizée Bidaut,22,1.74,Saint-Genis-sur-Menthon,2023-09-23,,2024,False,0,0,0,False,None,30
432,Miss Roussillon,Élise Aquilina,21,1.71,Cabestany,2023-08-05,Top 15 (11e)\nPrix de l’académie des Miss[25],2024,True,6,6,12,False,None,31


## Remmaping régions

certains comités ont changé de noms

In [147]:
df_miss['ID_REGION'].unique()

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36], dtype=int64)

In [148]:
# Regrouper les régions par ID_REGION
region_groups = df_miss.groupby('ID_REGION')['Région'].unique()

# Afficher les groupes
for id_region, regions in region_groups.items():
    print(f"ID_REGION: {id_region}")
    print(f"Régions associées : {regions}\n")

ID_REGION: 1
Régions associées : ['Miss Alsace']

ID_REGION: 2
Régions associées : ['Miss Aquitaine']

ID_REGION: 3
Régions associées : ['Miss Auvergne']

ID_REGION: 4
Régions associées : ['Miss Bourgogne']

ID_REGION: 5
Régions associées : ['Miss Bretagne']

ID_REGION: 6
Régions associées : ['Miss Centre']

ID_REGION: 7
Régions associées : ['Miss Champagne-Ardenne']

ID_REGION: 8
Régions associées : ['Miss Corse']

ID_REGION: 9
Régions associées : ["Miss Côte d'Azur"]

ID_REGION: 10
Régions associées : ['Miss Franche-Comté']

ID_REGION: 11
Régions associées : ['Miss Guadeloupe']

ID_REGION: 12
Régions associées : ['Miss Guyane']

ID_REGION: 13
Régions associées : ['Miss Île-de-France']

ID_REGION: 14
Régions associées : ['Miss Languedoc-Roussillon']

ID_REGION: 15
Régions associées : ['Miss Limousin']

ID_REGION: 16
Régions associées : ['Miss Lorraine']

ID_REGION: 17
Régions associées : ['Miss Martinique']

ID_REGION: 18
Régions associées : ['Miss Mayotte']

ID_REGION: 19
Régions ass

In [150]:
## Compter le nombre de lignes par ID_REGION
id_region_counts = df_miss['ID_REGION'].value_counts()

# Filtrer les ID_REGION avec moins de 13 occurrences
potential_remappings = id_region_counts[id_region_counts < 13]

# Afficher les résultats avec les régions associées
print("Comptage des ID_REGION avec noms des régions associées :\n")
for id_region, count in potential_remappings.items():
    regions = df_miss[df_miss['ID_REGION'] == id_region]['Région'].unique()
    print(f"ID_REGION: {id_region}")
    print(f"Nombre d'occurrences : {count}")
    print(f"Régions associées : {regions}\n")

# Pour information complète, on peut afficher aussi les ID_REGION ayant 13 occurrences ou plus
print("\nID_REGION bien représentés (13 occurrences ou plus) :\n")
for id_region, count in id_region_counts.items():
    if count >= 13:
        regions = df_miss[df_miss['ID_REGION'] == id_region]['Région'].unique()
        print(f"ID_REGION: {id_region}")
        print(f"Nombre d'occurrences : {count}")
        print(f"Régions associées : {regions}\n")

Comptage des ID_REGION avec noms des régions associées :

ID_REGION: 35
Nombre d'occurrences : 9
Régions associées : ['Miss Centre-Val-de-Loire']

ID_REGION: 31
Nombre d'occurrences : 8
Régions associées : ['Miss Roussillon']

ID_REGION: 34
Nombre d'occurrences : 6
Régions associées : ['Miss Saint-Martin et Saint-Barthélemy']

ID_REGION: 23
Nombre d'occurrences : 5
Régions associées : ['Miss Orléanais']

ID_REGION: 25
Nombre d'occurrences : 5
Régions associées : ['Miss Pays de Savoie']

ID_REGION: 6
Nombre d'occurrences : 5
Régions associées : ['Miss Centre']

ID_REGION: 32
Nombre d'occurrences : 5
Régions associées : ['Miss Saint-Pierre-et-Miquelon']

ID_REGION: 36
Nombre d'occurrences : 1
Régions associées : ['Miss Wallis et Futuna']


ID_REGION bien représentés (13 occurrences ou plus) :

ID_REGION: 1
Nombre d'occurrences : 14
Régions associées : ['Miss Alsace']

ID_REGION: 15
Nombre d'occurrences : 14
Régions associées : ['Miss Limousin']

ID_REGION: 30
Nombre d'occurrences : 14
Ré

In [151]:
# 1. Miss Orléanais et Miss Centre
centre_val_de_loire_id = df_miss.loc[df_miss['Région'] == 'Miss Centre-Val-de-Loire', 'ID_REGION'].iloc[0]
df_miss.loc[df_miss['Région'].isin(['Miss Orléanais', 'Miss Centre']), 'ID_REGION'] = centre_val_de_loire_id

# 2. Miss Pays de Savoie
rhones_alpes_id = df_miss.loc[df_miss['Région'] == 'Miss Rhône-Alpes', 'ID_REGION'].iloc[0]
df_miss.loc[df_miss['Région'] == 'Miss Pays de Savoie', 'ID_REGION'] = rhones_alpes_id

# 3. Miss Saint-Pierre-et-Miquelon et Miss Saint-Martin et Saint-Barthélemy
new_id_saint_martin = 37  # Un nouvel ID_REGION pour ces deux régions
df_miss.loc[df_miss['Région'].isin(['Miss Saint-Pierre-et-Miquelon', 'Miss Saint-Martin et Saint-Barthélemy']), 'ID_REGION'] = new_id_saint_martin

# Vérification des changements
updated_regions = df_miss[['Région', 'ID_REGION']].drop_duplicates().sort_values('ID_REGION')
print(updated_regions)

                                    Région  ID_REGION
0                              Miss Alsace          1
1                           Miss Aquitaine          2
2                            Miss Auvergne          3
3                           Miss Bourgogne          4
4                            Miss Bretagne          5
6                   Miss Champagne-Ardenne          7
7                               Miss Corse          8
8                         Miss Côte d'Azur          9
9                       Miss Franche-Comté         10
10                         Miss Guadeloupe         11
11                             Miss Guyane         12
12                      Miss Île-de-France         13
13               Miss Languedoc-Roussillon         14
14                           Miss Limousin         15
15                           Miss Lorraine         16
16                         Miss Martinique         17
17                            Miss Mayotte         18
18                      Miss

# Export data

In [152]:
file_path = "../data"
if os.path.exists(file_path)==False:
    os.makedirs(file_path, exist_ok=True)

file_name='data_miss_2011_2024.csv'
path_output=f'{file_path}/{file_name}'
df_miss.to_csv(path_output, index=False)